In [2]:
import pandas as pd
import numpy as np
import re
# from unidecode import unidecode
import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en_core_web_md', disable=["ner", "textcat", "entity_ruler", "merge_noun_chunks", "merge_entities", "merge_subtokens"])

In [3]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

In [5]:
dfExoplanetsDoc=pd.read_pickle("/Users/gerdgrasshoff/Dropbox/temp/df_doc.pcl")
# dfExoplanetsDoc=pd.read_pickle("c:/Dropbox/temp/df_doc.pcl")

In [ ]:
len(dfExoplanetsDoc)

In [ ]:
sentences = []
for doc in dfExoplanetsDoc:
    sent = []
    for sent in doc.sents:
              sentences.append(sent.string.strip())

In [ ]:
len(sentences)

In [ ]:
dfExoplanetsDoc[40]

In [ ]:
def showsentence(nrdoc,nr):
    s=list(dfExoplanetsDoc[nrdoc].sents)
    return(s[nr])
def splitsent(doc,nr):
    return(list(doc.sents)[nr])
def lsent(doc):
    return(list(doc.sents))

In [ ]:
s=lsent(dfExoplanetsDoc[10])
s

# Dependency label
https://github.com/clir/clearnlp-guidelines/blob/master/md/specifications/dependency_labels.md

https://stackoverflow.com/questions/40288323/what-do-spacys-part-of-speech-and-dependency-tags-mean/40288324

https://nlp.stanford.edu/software/dependencies_manual.pdf

In [ ]:
satz=s[1]
satz

In [ ]:
displacy.render(nlp(satz), style="dep")

In [ ]:
[(t.text,t.pos_,t.dep_,t.head.text,t.head.pos_,[c for c in t.children]) for t in satz]

In [ ]:
def phil_satz(satz):
    merkmal = dict({"satz":satz.text})
    for t in satz:
        if t.dep_ == "ROOT":
            r=t.head.text
            merkmal.update({"act":r})
#    print("root:",merkmal["act"])
    for t in satz:
#        print("CHECK",t.text,"             ATT=",t.dep_,t.head.text,t.head.pos_)
        if t.dep_ == "dobj" and t.head.pos_ == "VERB":
            merkmal.update({"obj":''.join(w.text_with_ws for w in t.subtree)})
        if t.dep_ == "nsubj" and merkmal["act"]==t.head.text:
            merkmal.update({"subject":t.text.lower()})
        if t.dep_ in ("ccomp"):
            merkmal.update({"ccomp":''.join(w.text_with_ws for w in t.subtree)})
        if t.dep_ in ("acomp"):
            merkmal.update({"acomp":''.join(w.text_with_ws for w in t.subtree)})

    return(merkmal)
phil_satz(satz)

In [ ]:
l=[]
df=pd.DataFrame(columns=["satz","subject","act","ccomp","acomp"])

In [ ]:
for i in range(1000):
    s=sentences[i]
    p=phil_satz(nlp(s))
    df=df.append(p,ignore_index=True)

In [ ]:
pd.set_option('max_colwidth', 260)
df.head(20)

In [ ]:
dfs=df[df["subject"]=="we"]
dfs.head(4)

In [ ]:
dfAct=df.groupby(["subject","act"]).size().sort_values(ascending=False).reset_index()
dfAct[dfAct["subject"]=="we"]